In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

import torch.optim as optim

In [2]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

trainset = torchvision.datasets.CIFAR10(root = '../data',train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size= 32, shuffle=False, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform= transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=0)

classes = ('plane','car','bird','cat','deer','dog','frong','horse','ship','truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class BasicBlock(nn.Module):
    def __init__(self, inchannel, channel, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(inchannel, channel, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channel)
        self.conv2 = nn.Conv2d(channel, channel, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(channel)
        self.downsample = downsample
        
    def forward(self, x):
        shortcut = x
        
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        
        if self.downsample is not None:
            shortcut = self.downsample(x)
            
        out += shortcut
        out = F.relu(out)
        
        return out

In [4]:
class ResNet(nn.Module):
    def __init__(self,num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 16
        self.conv1 = nn.Conv2d(3,16,kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self.make_layer(BasicBlock, 16, 3)
        self.layer2 = self.make_layer(BasicBlock, 32, 3, stride=2)
        self.layer3 = self.make_layer(BasicBlock, 64, 3, stride=2)
        self.avgpool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, channel, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inchannel != channel:
            downsample = nn.Sequential(nn.Conv2d(self.inchannel, channel, kernel_size=1, stride=stride, bias=False),
                                      nn.BatchNorm2d(channel),)
            
        layers = []
        layers.append(block(self.inchannel, channel, stride, downsample))
        self.inchannel = channel
        for i in range(1, blocks):
            layers.append(block(self.inchannel, channel))

        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        x = self.avgpool(x)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        
        return x

In [5]:
net = ResNet()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if torch.cuda.device_count() > 1:
    print("Let's use ",torch.cuda.device_count(),"GPUs!")
    net = nn.DataParallel(net)
    
net.to(device)

Let's use  2 GPUs!


DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=0.1, weight_decay= 0.0001, momentum=0.9)

In [7]:
def train(epoch):
    net.train()
    for batch_idx,(imgs, labels) in enumerate(trainloader):
        imgs, labels = imgs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = net(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(imgs), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.data[0]))
            
    print("Training finished")

In [8]:
def test():
    net.eval()
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            c = (predicted == labels).squeeze()
        
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    total_correct =  0
    for i in range(10):
        total_correct += 100*class_correct[i]/class_total[i]
        print('Accuracy of %5s : %2d %%'%(classes[i], 100*class_correct[i]/class_total[i]))

    print('Accuracy overall: %d' % (total_correct/10))
    
    """
    with torch.no_grad():
        for imgs, labels in testloader:
            imgs,labels = imgs.to(device), labels.to(device)

            outputs = net(imgs)
            test_loss += criterion(outputs,labels).data[0]
            pred = outputs.data.max(1, keepdim=True)[1]
            correct += pred.eq(labels.data.view_as(pred)).cpu().sum()

        test_loss /= len(testloader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(testloader.dataset), 100. * correct / len(testloader.dataset)))
    """

In [9]:
for epoch in range(1,10):
    train(epoch)
    test()

/home/hsue/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  from ipykernel import kernelapp as app


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.418133
Train Epoch: 1 [32000/50000 (64%)]	Loss: 1.668473
Training finished
Accuracy of plane : 23 %
Accuracy of   car : 77 %
Accuracy of  bird : 38 %
Accuracy of   cat : 47 %
Accuracy of  deer : 31 %
Accuracy of   dog : 18 %
Accuracy of frong : 64 %
Accuracy of horse : 75 %
Accuracy of  ship : 83 %
Accuracy of truck : 57 %
Accuracy overall: 51
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.322467
Train Epoch: 2 [32000/50000 (64%)]	Loss: 1.179480
Training finished
Accuracy of plane : 58 %
Accuracy of   car : 82 %
Accuracy of  bird : 51 %
Accuracy of   cat : 45 %
Accuracy of  deer : 57 %
Accuracy of   dog : 44 %
Accuracy of frong : 74 %
Accuracy of horse : 80 %
Accuracy of  ship : 84 %
Accuracy of truck : 84 %
Accuracy overall: 66
Train Epoch: 3 [0/50000 (0%)]	Loss: 0.960593
Train Epoch: 3 [32000/50000 (64%)]	Loss: 1.090334
Training finished
Accuracy of plane : 59 %
Accuracy of   car : 86 %
Accuracy of  bird : 58 %
Accuracy of   cat : 59 %
Accuracy of  deer :